In [1]:
import os
import re
import upload_v2 as v2

In [2]:
dict_refer = v2.dict_refer
comp = v2.comp

In [3]:
files = ["20408242 박영덕-가족관계, 제적등본외서류.pdf"]

for f in files:
    print("-----------------------------------------")
    name_items = {"key" :"", "name" :"", "docu" :"", "event" :"", "extra" :"", "date" :""}
    name_extra = None
    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    name_items["key"], n = v2.keyFnc(n)
    print("afterkeyF n = ", n)
    isEvent, docuEventNameextra = v2.eventFnc(n) ### docu, event
    print("isEvent", isEvent)
    if isEvent == 2 :
        print("경정사건인데 nodocu")
        continue
    elif isEvent == 1 : # 정상
        name_items["docu"] = docuEventNameextra[0] #사건번호>>docu 할당 #############
        name_items["event"] = docuEventNameextra[1]
        name_extra = docuEventNameextra[2]
    else : # 사건번호 없음
        isDocu, docuNameExtra = v2.setDocu(n)
        print("isDocu : ", isDocu, "docuNameExtra : ", docuNameExtra)
        if isDocu :
            name_items["docu"] = docuNameExtra[0]#비사건번호>>docu 할당 #############
            name_extra = docuNameExtra[1]
        else : # nodocu
            print("setDocu 결과 nodocu")
            continue
    date, temp = v2.dateFnc(name_extra) ### date, name, extra
    print(f"date : {date}, temp = {temp}")
    name, extra = v2.rmNeedlessSharp(temp)   
    print(f"name = {name}, extra={extra}")
    if len(name) < 2 : 
        try :
            name = dict_refer[name_items["key"]][2]
        except :
            print("이름을 찾는데 키가 없어.")
        
        for ptrn in re.findall("[가-힣a-zA-Z]+", name) : # 새로 가져온 name에만 적용
            extra = re.sub(ptrn+"의?", "", extra) #다행히 ')의'는 '~~(주)의'밖에 안 보이네. 혹시 하다가 예외 많으면 \b의\b를 따로 해주는 것으로
        
        name, extra = v2.rmNeedlessSharp(name+"#"+extra) #list를 반환하기 때문에 
    
    name = v2.rm_s(name)
    extra = v2.rm_s(extra)

    name_items["name"] = name #### name, extra, date 할당 ######################
    name_items["extra"] = extra
    name_items["date"] = date
    new_f = "_".join(filter(lambda x :bool(x), name_items.values())) + ext
    print(f, ">>>", new_f)
    

-----------------------------------------
afterkeyF n =  박영덕-가족관계, 제적등본외서류
isEvent 0
isDocu :  True docuNameExtra :  ['등', '박영덕-가족관계, 제적#등본외서류']
date : , temp = 박영덕-가족관계, 제적#등본외서류
name = 박영덕 가족관계 제적, extra=등본외서류
20408242 박영덕-가족관계, 제적등본외서류.pdf >>> 20408242_박영덕 가족관계 제적_등_등본외서류.pdf


In [16]:
p = re.compile(r"신분증|(기초)?수급자?|차상위|(법인|사용)?\s?인감|기본\s?증명서|(가족|혼인)\s?관계|이혼|입양|친양자|졸업|병적"),
f = "20408242_박영덕 가족관계 제적_등본_외서류.pdf"
p.search(f)

In [4]:
subStem = "ghdrlfehd 3월일 초본"
# p_y = re.compile(r'(\d\d|\d\d\d\d)년')
# p_m = re.compile(r'(\d{1,2})월')
# p_d = re.compile(r'(\d{1,2})일')
# if p_y.search(subStem) or p_m.search(subStem) or p_d.search(subStem) :
#     print("good")
#     d = ""
#     py = p_y.search(subStem)
#     pm = p_m.search(subStem)
#     pd = p_d.search(subStem)
#     if py : 
#         d = py[1]
#         subStem = p_y.sub("", subStem)
#     if pm :
#         if len(pm[1]) == 1 : d = d + "0" + pm[1]
#         else : d = d + pm[1]
#         subStem = p_m.sub("", subStem)
#     if pd :
#         if len(pd[1]) == 1 : d = d + "0" + pd[1]
#         else : d = d + pd[1]
#         subStem = p_d.sub("", subStem)
#         print("yes", d, subStem)
v2.dateFnc(subStem)

('03', 'ghdrlfehd 일 초본')

In [3]:
import re
f = "20411631 세목별과세증명.pdf"
p = re.compile(r"기타|집행문\s?부여의\s?소|배송[가-힣]+|(채권|양도|양수|매매).*계약|화해(?!권고)|분할|상환|감면|(상속)?\s?한정\s?승인|상속\s?포기|지방세|세목별|과세") # 제거가 아니므로 첫 글자만 잘 찾으면 된다.p.search("20433506 김상진 주민등록정보_(1).pdf")
m = p.search(f)
print(f[:m.start()])
print(f[m.start():])


20411631 
세목별과세증명.pdf


In [14]:
a = ""
print(a[:-1])